<a href="https://colab.research.google.com/github/786aafreen/Data-Analysis/blob/main/DAta_Analytics_Asssgn_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For the dataset given

1. Comment on the nature of the data using suitable
visualization techniques
2. Check the stationary using ADF test
and autocorrelation plot. 
3. Forecast the target variable prediction using a suitable
type of model.  
4. Evaluate the different types of error residues to
check fitness of good of the  model.  
5. Forecast the predictions for next 5 years on  target variable.

**Forecasting** is a technique that uses historical data as inputs to make informed estimates that are predictive in determining the direction of future trends. Businesses utilize forecasting to determine how to allocate their budgets or plan for anticipated expenses for an upcoming period of time.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/monthly_milk_production (1).csv',  parse_dates = True, squeeze = True)

In [ ]:
df.tail()

**Why should time series be stationary?**

Stationary series is easier for statistical models to predict effectively and precisely.

1. Comment on the nature of the data using suitable
visualization techniques

How to check stationarity?

**Visualizations**

The most basic methods for stationarity detection rely on plotting the data, and visually checking for trend and seasonal components.

In [ ]:
# Load the libraries
import numpy as np
import pandas as pd
# Load Statsmodels 
import statsmodels.api as sm
# Load Matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Check the dimensionality of the dataset
df.shape

In [ ]:
df.head()

In [ ]:
print("Dataset has {} records and {} columns".format(df.shape[0], df.shape[1]))

In [ ]:
plt.figure(figsize=(8,8))
df.plot()
plt.ylabel('Production', color='red')
plt.title('Milk Production (Monthly data)', color='purple')
plt.grid(True)
plt.show()

**Time-Series ::**

A time series is a series of measurements on the same variable collected over time. These measurements are made at regular time intervals.A time series is a series of data points indexed in time order. Most commonly, a time series is a sequence taken at successive equally spaced points in time. Thus it is a sequence of discrete-time data.

In [ ]:
timeseries = df['Production']

In [ ]:
timeseries.rolling(12).mean().plot(label='12 Month Rolling Mean')
timeseries.rolling(12).std().plot(label='12 Month Rolling Std')
timeseries.plot(figsize = (12,8))
plt.legend()

**Time-Series Prediction ::**

Here we are predicting historical data of past 15 years of Milk Production.

We can see above clearly through graph Production continuously increasing.

In [ ]:
timeseries.rolling(12).mean().plot(label='12 Month Rolling Mean')
timeseries.plot(figsize = (12,8))
plt.legend();

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(df['Production'], freq=12)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

**Note::** We have used only one variable here , MilkProduction (the MilkProduction of the past 15 years).

Hence this is called as the Univariate Time Series Analysis/Forecasting.

2. Check the stationary using ADF test
and autocorrelation plot.


# Testing for Stationarity
# We can use the Augmented Dickey-Fuller unit root test.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
result = adfuller(df['Production'])

In [ ]:
result

In [ ]:
df.set_index('Date',inplace=True)

In [ ]:
print('Augmented Dickey-Fuller Test:')
labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']

for value,label in zip(result,labels):
    print(label+' : '+str(value) )
    
if result[1] <= 0.05:
    print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
else:
   print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
# Store in a function!
def adf_check(time_series):
    """
    Pass in a time series, returns ADF report
    """
    result = adfuller(time_series)
    print('Augmented Dickey-Fuller Test:')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):

      print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

#So afterthat now we realized that our data is seasonal (it is also pretty obvious from the plot itself). 
# This means we need to use Seasonal ARIMA on our model. If our data was not seasonal, it means we could use just ARIMA on it.

In [ ]:
# First Difference
df['Production First Difference'] = df['Production'] - df['Production'].shift(1)

In [ ]:
adf_check(df['Production First Difference'].dropna())

In [ ]:
df['Production First Difference'].plot()

In [ ]:
# Second Difference
# Sometimes it would be necessary to do a second difference 
# This is just for show, we didn't need to do a second difference in our case
df['Production Second Difference'] = df['Production First Difference'] - df['Production First Difference'].shift(12)

In [ ]:
adf_check(df['Production Second Difference'].dropna())

In [ ]:
df['Production Second Difference'].plot()

Seasonal Difference

In [ ]:
df['Seasonal Difference'] = df['Production']-df['Production'].shift(12)
df['Seasonal Difference'].plot()

# Seasonal Difference by itself was not enough!

In [ ]:
# Seasonal Difference by itself was not enough!
adf_check(df['Seasonal Difference'].dropna())

In [ ]:
df['Seasonal First Difference'] = df['Production First Difference'] - df['Production First Difference'].shift(12)

In [ ]:
df['Seasonal First Difference'].plot()

In [ ]:
adf_check(df['Seasonal First Difference'].dropna())

# Autocorrelation and Partial Autocorrelation Plots

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig_prod_first = plot_acf(df['Production First Difference'].dropna())

In [ ]:
fig_seasonal_first = plot_acf(df['Seasonal First Difference'].dropna())

In [ ]:
 #Pandas also has this functionality built in, but only for ACF, not PACF.
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(df['Seasonal First Difference'].dropna())

# Partial Autocorrelation

In [ ]:
result=plot_acf(df['Seasonal First Difference'].dropna())

# Final ACF and PACF Plots

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['Seasonal First Difference'].iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['Seasonal First Difference'].iloc[13:], lags=40, ax=ax2)

# Using the Seasonal ARIMA model

In [ ]:
 #For non-seasonal data
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# We have seasonal data!
model = sm.tsa.statespace.SARIMAX(df['Production'],order=(0,1,0), seasonal_order=(1,1,1,12))
results = model.fit()
print(results.summary())

In [ ]:
results.resid.plot()

In [ ]:
results.resid.plot(kind='kde')

In [ ]:
df.head()

# Prediction of Future Values

In [ ]:
df['forecast'] = results.predict(start = 150, end= 168, dynamic= True)  
df[['Production','forecast']].plot(figsize=(12,8))

In [ ]:
df['forecast'] = results.predict(start = 168, end = 188, dynamic= True)  
df[['Production', 'forecast']].plot(figsize=(12, 8)) 